# Controllable loop process
> iteration with callbacks

In [1]:
# default_exp loop

In [1]:
# export 
import numpy as np

In [2]:
# export
from tqdm import tqdm
from types import MethodType
import inspect
from functools import partial

def method4all(f):
    """
    Use this function as a decorator,
    The decorated function under Loop class can be used on outer layer
    """
    setattr(f,"forall",True)
    return f

class StorageCore:
    def __init__(self,start_layer):
        self.layers = []
        self.lmap = dict()
        self.forall_pool = dict()
        self.new_layer(start_layer)
        self.i = -1
        self.epoch = -1
    
    def new_layer(self,layer):
        layer.core = self
        self.layers.append(layer)
#         self.seq_layers(self.layers)
        self.lmap[layer._loop_layer]=layer
        if hasattr(self,layer.name):
            raise KeyError(f"Layer name already exist: {layer.name}")
        setattr(self,layer.name,layer)
        self.update_forall(layer)
        self.assign_forall(layer)
  
    def __repr__(self):
        return str(self.lmap)
        
    def for_all_functions(self,obj):
        return dict(inspect.getmembers(obj, 
                  predicate=lambda x:hasattr(x,"forall")))
    
    def update_forall(self,obj):
        self.forall_pool.update(self.for_all_functions(obj))
        
    def assign_forall(self,obj):
        for name,f in self.forall_pool.items():
            setattr(obj,name,f)

class Loop:
    """
    Basic loop class
    """
    _loop_layer = 0.
    def __init__(self,iterable = [],name=None):
        self.iterable  = iterable
        self.next_layer = None
        self.name = name if name!=None else self.__class__.__name__
        
        if hasattr(iterable,"_loop_layer"):
            self._loop_layer = iterable._loop_layer + 1
            iterable.next_layer = self
            iterable.core.new_layer(self)
        else:
            self.core = StorageCore(self)
        
    def __call__(self):
        pass
    
    def __repr__(self):
        return f"layer:>>>{self.name}"
    
    def on(self,iterable):
        self.iterable = iterable
        self._loop_layer = iterable._loop_layer + 1
        
    def func_detail(self,f):
        detail = dict({"🐍func_name":f.__name__,
                       "⛰doc":f.__doc__,
                       "😝var":",".join(f.__code__.co_varnames),
                       "😜names":",".join(f.__code__.co_names),
                      })
        return detail
    
    def summary(self):
        rt = f"Loop layer {self.name} summary:\n"
        rt+= "="*50+"\n"
        funcs = []
        for idx,layer in self.core.lmap.items():
            rt+= f"🍰layer{idx}\t{str(layer)}\n"
            for fname,f in self.core.for_all_functions(layer).items():
                if id(f) not in funcs:
                    rt+="\t"
                    rt+="\n\t".join(f"[{k}]\t{v}" for k,v in self.func_detail(f).items())
                    rt+="\n\t"+"."*35+"\n"
                    funcs.append(id(f))
            rt+="-"*50+"\n"
        rt+= "="*50+"\n"
        print(rt)
    
    def __len__(self):
        return self.iterable.__len__()
    
    def run(self,):
        """
        Run through iteration
        run start_call for every layer on the start of iteration
            run __call__ for every layer for each iteration
        run end_call for every layer on the end of iteration
        """
        first = self.layers[0]
        self.refresh_i()
        first.start_callon()
        for element in first:
            first()
        first.end_callon()
        
    def refresh_i(self):
        self.core.i=-1
        self.core.epoch+=1
        
    def update_i(self):
        self.core.i+=1
        
    def callon(self):
        self()
        self.iter_cb()
            
    def start_callon(self):
        self.start_call()
        self.start_cb()
            
    def end_callon(self):
        self.end_call()
        self.end_cb()
            
    def iter_cb(self):
        """
        call back during each iteration
        """
        if self.next_layer!=None:
            self.next_layer.callon()
        self.after()
            
    def after(self):
        pass
            
    def start_cb(self):
        """
        callback at the start of iteration
        """
        if self.next_layer!=None:
            self.next_layer.start_callon()
    
    def end_cb(self):
        """
        callback at the end of iteration
        """
        if self.next_layer!=None:
            self.next_layer.end_callon()
            
    def start_call(self):
        pass
    
    def end_call(self):
        pass
            
    def __iter__(self,):
        for element in self.iterable:
            if self._loop_layer ==0:
                self.update_i()
            self.core.element = element
            self.callon()
            yield self.element
            
    def __getattr__(self,k):
        return getattr(self.core,k)
    
    def is_newloop(self):
        """
        return Bool:Is this a new loop ready to start
        """
        return (self.i==-1 or self.i==self.__len__()-1)

## Loop-in-loop concept
Loop and its inheriting descendants runs on iterable, incuding other ```Loop```

In [3]:
l = Loop(range(5,10),name = "hahahha")
l

layer:>>>hahahha

In [4]:
class Printer(Loop):
    def __call__(self):
        print(f"i_{self.core.i}",end="\t")
        print(self.core.element)
        
    def start_call(self):
        print(f"start of epoch {self.epoch}")
        
    def end_call(self):
        print(f"end of epoch {self.epoch}")
l2 = Printer(l)

In [5]:
l2.run()

start of epoch 0
i_0	5
i_1	6
i_2	7
i_3	8
i_4	9
end of epoch 0


## Progress bar

In [6]:
# export
class ProgressBar(Loop):
    def __init__(self,iterable=[],jupyter = True,mininterval = 1e-1):
        super().__init__(iterable,"Progressb Bar")
        
        if jupyter: # jupyter widget
            from tqdm.notebook import tqdm
            
        else: # compatible for console print
            from tqdm import tqdm
            
        self.tqdm = tqdm
        self.mininterval = mininterval
        self.data = dict()
        
    @method4all
    def pgbar_data(self,data):
        """
        update progress bar with python dictionary
        data: python dictionary
        """
        self.t.set_postfix(data)
    
    @method4all
    def pgbar_description(self,text):
        """
        update progress bar prefix with text string
        """
        self.t.set_description_str(f"{text}")
        
    def start_call(self):
        self.create_bar()
        
    def end_call(self):
        self.t.close()
        
    def __call__(self):
        self.t.update(1)
            
    def create_bar(self):
        self.t = self.tqdm(total=len(self.iterable),
                           mininterval=self.mininterval)

In [7]:
from tqdm.notebook import tqdm,tqdm_notebook

In [8]:
pb = ProgressBar(Loop(range(5,10)),jupyter = True)
pb.run()

### Error Handling

In cases you don't want error stop the iteration, meanwhile taking down all the error message

In [9]:
# export
class Tolerate(Loop):
    """
    Tolerate any error happened downstream
    layer2 = Tolerate(upstream_iterable)
    # build downstream tasks
    layer3 = OtherApplication(layer2)
    layer3.run()
    # show the happened error message
    layer3.error_list()
    """
    def __init__(self,iterable = []):
        super().__init__(iterable,)
        self.errors = list()
    
    @method4all
    def error_list(self):
        """
        A list of errors happend so far
        """
        return self.errors
        
    def callon(self):
        """
        Usual callon method like other loop, but tolerate all errors
        """
        try:
            # this will only capture the downstream error
            self.iter_cb()
        except Exception as e:
            self.errors.append(dict(stage="middle",i=self.i,epoch=self.epoch,error=e))
    
    def start_callon(self):
        try:
            self.start_cb()
        except Exception as e:
            self.errors.append(dict(stage="start",i=self.i,epoch=self.epoch,error=e))
            
    def end_callon(self):
        try:
            self.end_cb()
        except Exception as e:
            self.errors.append(dict(stage="end",i=self.i,epoch=self.epoch,error=e))

class LambdaCall(Loop):
    def __init__(self,iterable = [],func = lambda x:x):
        super().__init__(iterable,name=f"Lambda<{hex(id(self))}>")
        self.func = func
    
    def __call__(self):
        self.func(self)

In [10]:
l1 = ProgressBar(Loop(range(1,10)))
error_loop = LambdaCall(l1,func= lambda x:(5-x.element)**(-2))

Running this will yield error

```python
error_loop.run()
```

In [11]:
def error_on_purpose(x):
    if x.element>5:
        raise ValueError(f"value too small, {x.element}")

l1 = ProgressBar(Loop(range(1,10)))
# add the tolerate layer in between
l2 = Tolerate(l1)
# tolerate can only tolerate the erorr on downstream
tolerated_loop = LambdaCall(l2,func=error_on_purpose)
l2.run()

In [12]:
import pandas as pd
pd.DataFrame(l2.error_list())

,stage,i,epoch,error
0,middle,5,0,"value too small, 6"
1,middle,6,0,"value too small, 7"
2,middle,7,0,"value too small, 8"
3,middle,8,0,"value too small, 9"


In [13]:
l2.summary()

Loop layer Tolerate summary:
🍰layer0.0	layer:>>>Loop
--------------------------------------------------
🍰layer1.0	layer:>>>Progressb Bar
	[🐍func_name]	pgbar_data
	[⛰doc]	
        update progress bar with python dictionary
        data: python dictionary
        
	[😝var]	self,data
	[😜names]	t,set_postfix
	...................................
	[🐍func_name]	pgbar_description
	[⛰doc]	
        update progress bar prefix with text string
        
	[😝var]	self,text
	[😜names]	t,set_description_str
	...................................
--------------------------------------------------
🍰layer2.0	layer:>>>Tolerate
	[🐍func_name]	error_list
	[⛰doc]	
        A list of errors happend so far
        
	[😝var]	self
	[😜names]	errors
	...................................
--------------------------------------------------
🍰layer3.0	layer:>>>Lambda<0x107114d50>
--------------------------------------------------



### Application upon progress bar
The function ```pgbar_data``` is decorated under ```method4all```, hence the function can be used else where

In [14]:
from datetime import datetime
class Application(Loop):
    def __call__(self):
        # create some data
        loop_data = dict(epoch=self.epoch,i = self.i,
                         val = self.element,)
        # update data to progress bar
        self.pgbar_data(loop_data)
        self.pgbar_description(str(datetime.now().strftime("%H:%M:%S")))

l3 = Application(ProgressBar(Loop(range(5,10)),jupyter=True))
l3.run()

## Print out the summary of the loop

In [15]:
l3.summary()

Loop layer Application summary:
🍰layer0.0	layer:>>>Loop
--------------------------------------------------
🍰layer1.0	layer:>>>Progressb Bar
	[🐍func_name]	pgbar_data
	[⛰doc]	
        update progress bar with python dictionary
        data: python dictionary
        
	[😝var]	self,data
	[😜names]	t,set_postfix
	...................................
	[🐍func_name]	pgbar_description
	[⛰doc]	
        update progress bar prefix with text string
        
	[😝var]	self,text
	[😜names]	t,set_description_str
	...................................
--------------------------------------------------
🍰layer2.0	layer:>>>Application
--------------------------------------------------



In [16]:
l3.run()

## Event

Event allows add-hoc callbacks to be created

In [169]:
# export 
class Event(Loop):
    """
    An event is the landmark with in 1 iteration
    """
    def __init__(self,iterable=[],event_name = None):
        super().__init__(iterable,event_name)
        self.event_name = event_name
        self.cbs = []
        self.create_cb_deco("on")
        self.create_cb_deco("before_1st")
        self.create_cb_deco("after_last")
    
    def create_cb_deco(self,moment):
        event_name = self.event_name
        wraper = getattr(self,moment).__func__
        wraper.__name__ = f"{moment}_{event_name}"
        wraper.__doc__ = f"""
            Append new {moment} callback for event:{event_name}
            Use this function as decorator
        """
        setattr(self,wraper.__name__,MethodType(method4all(wraper),self))
        
    def __call__(self):
        if len(self.cbs)>0:
            self.cbs[0].callon()
            
    def start_call(self):
        if len(self.cbs)>0:
            self.cbs[0].start_callon()
    
    def end_call(self):
        if len(self.cbs)>0:
            self.cbs[0].end_callon()
            
    def on(self,f):
        class EventCB1(Loop):
            def __init__(self_,iterable=[]):
                super().__init__(iterable=iterable,
                                 name = f"ON_{self.event_name}_{self.cbs.__len__()+1}_{f.__name__}")
                self_.f = f
                
            def __call__(self_):
                self_.f(self)
        new_cb = EventCB1()
        self.new_event_cb(new_cb)
        return f
    
    def before_1st(self,f):
        class EventCB2(Loop):
            def __init__(self_,iterable=[]):
                super().__init__(iterable=iterable,
                                 name = f"BEFORE_1ST_{self.event_name}_{self.cbs.__len__()+1}_{f.__name__}")
                self_.f = f
                
            def start_call(self_):
                self_.f(self)
        new_cb = EventCB2()
        self.new_event_cb(new_cb)
        return f
    
    def after_last(self,f):
        class EventCB3(Loop):
            def __init__(self_,iterable=[]):
                super().__init__(iterable=iterable,
                                 name = f"AFTER_LAST_{self.event_name}_{self.cbs.__len__()+1}_{f.__name__}")
                self_.f = f
                
            def end_call(self_):
                self_.f(self)
        new_cb = EventCB3()
        self.new_event_cb(new_cb)
        return f
                
    def new_event_cb(self,new_cb):
        new_cb.core.update_forall(new_cb)
        new_cb.core.assign_forall(new_cb)
        new_cb.core = self.core
        if len(self.cbs)>0:
            self.cbs[-1].next_layer = new_cb
#             self.cbs[-1].new_layer(new_cb)
        self.cbs.append(new_cb)

In [170]:
class ToString(Loop):
    def __call__(self,):
        self.core.element = f"variable_{self.i}"

class ChangeString(Event):
    def __init__(self,iterable=[]):
        super().__init__(iterable=iterable,event_name="CHANGE_STRING")

In [171]:
def get_frame(li):
    layer1 = ProgressBar(Loop(li,"base"))
    layer2 = ToString(layer1,)
    layer3 = ChangeString(layer2)
    # you can use this "on" syntax
    @layer3.CHANGE_STRING.on
    def trim(self):
        self.core.element = f"Var_{self.element}"
        
    # or this syntax
    @layer3.on_CHANGE_STRING
    def update(self):
        self.pgbar_description(self.element)
        
    @layer3.before_1st_CHANGE_STRING
    def print_start(self):
        """
        decorated
        """
        print(f"start_{self.epoch}_{self.i}")
        print(self.name)
    return layer3

In [172]:
l = get_frame(range(10))

In [173]:
l.core

{0.0: layer:>>>base, 1.0: layer:>>>Progressb Bar, 2.0: layer:>>>ToString, 3.0: layer:>>>CHANGE_STRING}

In [174]:
l.cbs

[layer:>>>ON_CHANGE_STRING_1_trim,
 layer:>>>ON_CHANGE_STRING_2_update,
 layer:>>>BEFORE_1ST_CHANGE_STRING_3_print_start]

In [175]:
l.summary()

Loop layer CHANGE_STRING summary:
🍰layer0.0	layer:>>>base
--------------------------------------------------
🍰layer1.0	layer:>>>Progressb Bar
	[🐍func_name]	pgbar_data
	[⛰doc]	
        update progress bar with python dictionary
        data: python dictionary
        
	[😝var]	self,data
	[😜names]	t,set_postfix
	...................................
	[🐍func_name]	pgbar_description
	[⛰doc]	
        update progress bar prefix with text string
        
	[😝var]	self,text
	[😜names]	t,set_description_str
	...................................
--------------------------------------------------
🍰layer2.0	layer:>>>ToString
--------------------------------------------------
🍰layer3.0	layer:>>>CHANGE_STRING
	[🐍func_name]	after_last_CHANGE_STRING
	[⛰doc]	
            Append new after_last callback for event:CHANGE_STRING
            Use this function as decorator
        
	[😝var]	self,f,EventCB3,new_cb
	[😜names]	Loop,new_event_cb
	...................................
	[🐍func_name]	after_last_CHANGE_STRING

In [176]:
l

layer:>>>CHANGE_STRING

In [177]:
l.layers

[layer:>>>base,
 layer:>>>Progressb Bar,
 layer:>>>ToString,
 layer:>>>CHANGE_STRING]

In [178]:
testcb = l.cbs[0]

In [179]:
testcb

layer:>>>ON_CHANGE_STRING_1_trim

In [180]:
l.run()

start_0_-1
CHANGE_STRING



In [181]:
layer = l.layers[0]
print(layer)
while layer.next_layer!=None:
    layer=layer.next_layer
    print(layer.start_call.__doc__)

layer:>>>base
None
None
None
